## Exploration of SQL files

### Programmatically download the SQL dumps

Let's first look at the available versions of the database

In [22]:
import requests
from bs4 import BeautifulSoup

base_url = "https://dumps.wikimedia.org/simplewiki/"
# Get the text text response of the base page
index = requests.get(base_url).text
soup_index = BeautifulSoup(index, "html.parser")

# In the base index, there should be multiple <a> tag leading
# to different version of the database
dumps = [a["href"] for a in soup_index.find_all("a") if a.has_attr("href")]

dumps

['../',
 '20200520/',
 '20200601/',
 '20200620/',
 '20200701/',
 '20200720/',
 '20200801/',
 '20200820/',
 '20200901/',
 '20200920/',
 '20201001/',
 '20201020/',
 'latest/']

As we can see, the last page is `lastest/` and this link might contains files with different timestamp because it's updated incrementally when a dump is being processed.

As of the time this notebook is being written, `20201020/` is being generated so if we look into `latest/`, we would see some files were updated on `20201001/` and some other were updated on `20201020/`.

For simplicity, while the newest dump is being generated, let's hardcode the timestamp and we will change it to a dynamic variable later on production.

For a more complex hanlder, we can always look for `<li>` tag with the word "waiting", meaning the version is being processed and we can schedule the task some time later.


In [23]:
if "latest" not in dumps[-1]:
    print("Couldn't find the latest dump")
    exit

# Later on production, we will use this.
dump = dumps[-2]

# For now, let's use this.
dump = "20201001/"

Once we have that, we can now look into a specific version, exact all `<li>` tag with `class="file"` to get all the file links. We can also filter everything else out, except `SQL` file.

In [27]:
# Create dump url with the base and the latest timestamp
dump_url = base_url + dump

# Retrieve the dump page
dump_html = requests.get(dump_url).text
soup_dump = BeautifulSoup(dump_html, "html.parser")

# Search for SQL files
files = []
for file in soup_dump.find_all("li", {"class": "file"}):
    text = file.text
    if "sql" in text:
        files.append((text.split()[0], text.split()[1:]))

files_to_download = [file[0] for file in files]



In [28]:
files

[('simplewiki-20201001-image.sql.gz', ['5', 'KB']),
 ('simplewiki-20201001-category.sql.gz', ['539', 'KB']),
 ('simplewiki-20201001-user_groups.sql.gz', ['3', 'KB']),
 ('simplewiki-20201001-page_props.sql.gz', ['5.7', 'MB']),
 ('simplewiki-20201001-iwlinks.sql.gz', ['1.8', 'MB']),
 ('simplewiki-20201001-site_stats.sql.gz', ['796', 'bytes']),
 ('simplewiki-20201001-wbc_entity_usage.sql.gz', ['5.1', 'MB']),
 ('simplewiki-20201001-externallinks.sql.gz', ['27.2', 'MB']),
 ('simplewiki-20201001-change_tag_def.sql.gz', ['1', 'KB']),
 ('simplewiki-20201001-page_restrictions.sql.gz', ['21', 'KB']),
 ('simplewiki-20201001-pagelinks.sql.gz', ['58.2', 'MB']),
 ('simplewiki-20201001-babel.sql.gz', ['3', 'KB']),
 ('simplewiki-20201001-sites.sql.gz', ['20', 'KB']),
 ('simplewiki-20201001-page.sql.gz', ['18.8', 'MB']),
 ('simplewiki-20201001-user_former_groups.sql.gz', ['2', 'KB']),
 ('simplewiki-20201001-protected_titles.sql.gz', ['15', 'KB']),
 ('simplewiki-20201001-redirect.sql.gz', ['811', 'KB'])

Once we have determined the files needed to be downloaded, we can use `Keras`'s `get_file` utility which is extremely handy to download a file and save to disk.

In [32]:
from keras.utils import get_file
from os import path
import subprocess

# Directory where keras download the files
dataset_dir = os.path.join(os.getcwd(), "datasets")

data_paths = []
file_info = []

for file in files_to_download:
    path = os.path.join(dataset_dir, file)

    if not os.path.exists(path):
        print(f"Downloading {file} ...")
        data_paths.append(
            get_file(fname=file, origin=dump_url + file, cache_subdir=dataset_dir)
        )
        # Find the file size in MB
        file_size = os.stat(path).st_size / 1e6
        print(file_size)

        file_info.append((file, file_size))

    else:
        # If file exists, put in the list still, for later processing
        data_paths.append(path)
        file_size = os.stat(path).st_size / 1e6
        file_info.append((file.split("-")[-1], file_size))

for data_path in data_paths:
    subprocess.call(["gunzip", f"{data_path}"])

8192/5596 [===========================================] - 0s 0us/step
0.005596
557056/552897 [==============================] - 2s 3us/step
0.552897
8192/3175 [=============================================================================] - 0s 0us/step
0.003175
5955584/5954045 [==============================] - 11s 2us/step
5.954045
1843200/1837306 [==============================] - 6s 4us/step
1.837306
8192/796 [====================================================================================================================================================================================================================================================================================================================] - 0s 0us/step
0.000796
5349376/5342004 [==============================] - 10s 2us/step
5.342004
28483584/28475883 [==============================] - 15s 1us/step
28.475883
8192/2044 [==========================================================================================

After the step above, we should now have all the `.sql` files under the folder `datasets`.

In [ ]:
# Get the paths of all sql files
sql_data_paths = [item[:-3] for item in data_paths]

# Run mysql command import the files
# It's okay to expose the password since it's a Docker
# container running on our local machine
for sql_data_path in sql_data_paths:
    with open(sql_data_path) as f:
        command = [
            "mysql",
            f"--host=b",
            f"--user=root",
            f"--password=LocalPassword",
            f"--database=simplewiki",
        ]
        p = subprocess.Popen(
            command,
            stdin=f,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        output, error = p.communicate()
        if p.returncode != 0:
            # We can handle this better by sending an email or similar
            print(error)
            

Lastly, clean up the files

In [33]:
# Once done with the data import, clean things up.
os.system("rm datasets/*")
os.system("rm -rf datasets")

0

With that, we should now be able to refactor the code to run as a function of our application.